In [1]:
import pandas as pd

In [2]:
import os
if os.environ.get("AITUEDATA"):
    print('set')
    data_directory = os.environ['AITUEDATA']
else:
    print('not set')
    #Eusebio
#     data_directory = "C:/Users/joaquin/Desktop/mem_aitue/Datos/"
    #Mini-Eusebio
#     data_directory = "C:/Users/jquin/Desktop/Memoria de Titulo/Aitue/Datos/"
    #PC-Mem
    data_directory = "C:/Users/Joaquin/Desktop/mem_aitue/Datos/"

set


# 1.Select Data

In [3]:
cot_all = pd.read_excel(data_directory+"cotizaciones_all.xlsx")
neg_all = pd.read_excel(data_directory+"negocios_all.xlsx")

In [4]:
cot_all.columns

Index(['ID', 'Fecha Cotizacion', 'Medio', 'Tipo de Medio', 'Proyecto', 'Etapa',
       'Productos', 'Presencial', 'Remoto', 'Jefe Comercial',
       'Ejecutivo Comercial', 'Total Productos', 'Descuentos',
       'Valor Final Venta', 'RUT', 'Nombre', 'Apellido 1', 'Apellido 2',
       'Nombre Completo', 'Telefono', 'Celular', 'Direccion', 'Region',
       'Provincia', 'Comuna', 'Correo Electronico', 'Sexo', 'Estado Civil',
       'Rango Edad', 'Fecha Nacimiento', 'Tipo Cliente', 'Razon Social',
       'Giro', 'Nacionalidad', 'N° Grupo Familiar', 'Actividad', 'Cargo',
       'Situacion Laboral', 'Empleador', 'Antiguedad Laboral', 'Profesión'],
      dtype='object')

In [5]:
s_columns = ['ID','Nombre','Apellido 1','Apellido 2', 'Fecha Cotizacion', 'Medio', 'Proyecto', 'Etapa','Productos', 'Presencial', 'Jefe Comercial',
             'Ejecutivo Comercial',
             'Total Productos','Descuentos','Valor Final Venta', 'RUT', 'Nombre', 'Apellido 1', 'Apellido 2',
             'Nombre Completo', 'Telefono', 'Celular', 'Direccion', 'Region','Provincia', 'Comuna', 
             'Correo Electronico', 'Sexo', 'Estado Civil','Rango Edad', 'Fecha Nacimiento', 'Profesión','Actividad']

cot_mod = cot_all[s_columns]

In [6]:
# ruts de los clientes en negocio
neg_ids = neg_all['ID Cotizacion'].unique().tolist()

# cot_neg = cot_all.copy()
cot_mod['Negocio'] = cot_all['ID'].isin(neg_ids)
cot_mod[cot_mod['Negocio'] == True].shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


(2336, 34)

# 2.Clean Data

In [7]:
# variables a impiar y chequear

# Limpiar variable fecha de nacimiento de alguna manera
# Limpiar variable Sexo usando rutificador
# Limpar variables Direccion usando en lo posible el rutificador y otras fuentes
# Llenar mv en decuentos con 0

## 2.1.Rut
Arreglar Ruts, luego validarlos para aplicar data crawling en rutificador.
Si el rut esta mal escrito por ende no es valido, se prosigue a buscar el rut por el nombre y arreglar el rut.
Todo esto con el find de obtener el genero, direccion, latiyud y longitud del cotizante

In [8]:
import chile_rut
import re

def val_rut(x):
    x = str(x)
    if len(x) == 0:
        return False
    x = re.sub('[|!@#$]', '', x)
    
    if len(x.split('-')) >= 2:
        return chile_rut.validate_rut(x)
    else:
        x = x[:-1] + '-' + x[-1]
        return chile_rut.validate_rut(x)

def clean_rut(x):
    x = str(x)
    
    if len(x) == 0:
        return False
    x = re.sub('[|!@#$]', '', x)
    
    if len(x.split('-')) >= 2:
        return x
    else:
        x = x[:-1] + '-' + x[-1]
        return x
    
cot_mod['Clean Rut'] = [clean_rut(x) for x in cot_mod['RUT'].values]
cot_mod['Valid Rut'] = [val_rut(x) for x in cot_mod['Clean Rut'].values]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
#Eliminar indice 1704 y arreglar rut max boris
print('Ruts que hicieron negocios pero no son validos')
cot_mod[(cot_mod['Valid Rut'] == False) & (cot_mod['Negocio'] == True)][['Nombre','RUT','Negocio','Valid Rut','Clean Rut']]

Ruts que hicieron negocios pero no son validos


,Nombre,Nombre,RUT,Negocio,Valid Rut,Clean Rut
7390,max boris,max boris,14030274-6,True,False,14030274-6
7504,max boris,max boris,14030274-6,True,False,14030274-6
1704,Error,Error,15516786-8,True,False,15516786-8


In [10]:
# ver que ruts invalidos pueden ser rescatados o arreglados buscando por sus nombres
print("Ruts catalogados como invalidos, buscar por nombres")
ruts_iv = cot_mod[(cot_mod['Valid Rut'] == False)] 
print(ruts_iv.shape)

ruts_invalidos = []
for group, frame in  ruts_iv.groupby('Clean Rut'):
    ruts_invalidos.append(str(group))

s_ruts = pd.Series(ruts_invalidos)
s_ruts.to_csv(data_directory + "ruts_invalidos_scrap.csv")

# print(ruts_iv[['Nombre','Apellido 1','RUT','Clean Rut','Valid Rut']].head())

Ruts catalogados como invalidos, buscar por nombres
(69, 36)


In [11]:
bla = pd.read_csv(data_directory + "ruts_invalidos_scrap.csv",header=None,names=['0','Rut']).drop('0',axis=1)
bla['Rut'].describe()
# bla['Rut'] = [chile_rut.format_rut(str(x)) for x in bla['Rut'].values]
# for rut in bla['']
# bla.head()

count             44
unique            44
top       17896434-2
freq               1
Name: Rut, dtype: object

In [12]:
# for group, frame in  ruts_iv.groupby('RUT'):
#     print(group, frame[['Nombre','Apellido 1']])
#     print('')

In [13]:
print("Ruts arreglados, estaban mal escritos y luego fueron validos.")
df = cot_mod[(cot_mod['RUT'] != cot_mod['Clean Rut']) & (cot_mod['Valid Rut'] ==True)]
print(df.shape)
print(df[['Nombre','Apellido 1','Apellido 2','RUT','Clean Rut','Valid Rut']].head(10))

Ruts arreglados, estaban mal escritos y luego fueron validos.
(1, 36)
      Nombre     Nombre Apellido 1 Apellido 1 Apellido 2 Apellido 2  \
8  Valentina  Valentina  Fernandez  Fernandez          .          .   

         RUT   Clean Rut  Valid Rut  
8  19483862k  19483862-k       True  


In [14]:
print("Ruts arreglados, estaban mal escritos y luego fueron validos.")
df = cot_mod[(cot_mod['RUT'] != cot_mod['Clean Rut']) & (cot_mod['Valid Rut'] ==False)]
print(df.shape)
print(df[['Nombre','Apellido 1','Apellido 2','RUT','Clean Rut','Valid Rut']].head(10))

Ruts arreglados, estaban mal escritos y luego fueron validos.
(4, 36)
              Nombre          Nombre Apellido 1 Apellido 1 Apellido 2  \
3716           Mario           Mario     Concha     Concha     Concha   
8943    Jose Edgardo    Jose Edgardo    Caamaño    Caamaño   Chamorro   
9099   Dinora  Nilda   Dinora  Nilda      Bravo      Bravo      Cerda   
9574  Anegelo Felipe  Anegelo Felipe      Godoy      Godoy    Morales   

     Apellido 2         RUT   Clean Rut  Valid Rut  
3716     Concha   156778804  15677880-4      False  
8943   Chamorro     6180535    618053-5      False  
9099      Cerda     9559524    955952-4      False  
9574    Morales  |7321289-5   7321289-5      False  


In [15]:
print("Ruts que estaban bien escritos.")
df = cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut'])]
print(df.shape)

Ruts que estaban bien escritos.
(16587, 36)


In [16]:
def format_rut(x):
    rut = x.split('-')
    l_s = list(rut[0])
    r_l = l_s[::-1]
    new = []
    i = 0
    new.append(rut[1])
    new.append('-')
    for c in r_l:
        if i==3:
            new.append('.')
            i = 0
        new.append(c)
        i +=1
    rut = ''.join(new[::-1])
    return rut

In [17]:
print("Ruts que estaban bien escritos y son validos.")
df = cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut']) &(cot_mod['Valid Rut'] ==True)]
print(df.shape)

ruts_validos = []
for group, frame in  df.groupby('Clean Rut'):
    ruts_validos.append(str(group))

# print(ruts_invalidos)
ruts_validos= [format_rut(x) for x in ruts_validos]
cot_mod['Format Rut'] = [format_rut(x) for x in cot_mod['Clean Rut'].values]
s_ruts = pd.Series(ruts_validos)
s_ruts.to_csv(data_directory + "ruts_validos_scrap.csv")

Ruts que estaban bien escritos y son validos.
(16522, 36)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [18]:
format_rut('18144836-9')

'18.144.836-9'

In [19]:
print("Ruts que estaban bien escritos pero no son validos, buscar ruts por nombres.")
df = cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut']) &(cot_mod['Valid Rut'] ==False)]
print(df.shape)

Ruts que estaban bien escritos pero no son validos, buscar ruts por nombres.
(65, 37)


In [65]:
#Ruts y sus datos obtenidos de forma valida
ruts_scrapped = pd.read_csv('C:/Users/jquin/Desktop/Memoria de Titulo/Aitue/Data Crawling/rut/ruts_validos.csv')
ruts_scrapped['RUT'] = [x.replace(' ','') for x in ruts_scrapped['RUT'].values]
ruts_scrapped['RUT'] = [x.replace('K','k') for x in ruts_scrapped['RUT'].values]
print(ruts_scrapped.shape)
print(ruts_scrapped.head())

(6012, 5)
                                   Name           RUT Gender  \
0    SCHNEIDER BARRERA RICARDO ERNESTO   10.023.348-7   VAR    
1      MORALES ESPINOZA CLAUDIO ANDRES   10.020.262-k   VAR    
2   SALGADO VALENZUELA ALFREDO EDUARDO   10.019.023-0   VAR    
3          MEIER PUCHEU SERGIO RODRIGO   10.016.093-5   VAR    
4      TEMPIO ETCHEVERRY MAURO RODRIGO   10.015.654-7   VAR    

                             Adress                 Comuna  
0                J  CISTERNAS 1612              LA SERENA   
1   ALEJANDRO SEXTO 2787LZO ARENAS      CONCEPCION CENTRO   
2         ANDALIEN 395 CHILLANCITO      CONCEPCION CENTRO   
3            RALUN S/NLONCO PARQUE            CHIGUAYANTE   
4                  LOS AVELLANOS 8    SAN PEDRO DE LA PAZ   


In [21]:
cot_mod.columns

Index(['ID', 'Nombre', 'Apellido 1', 'Apellido 2', 'Fecha Cotizacion', 'Medio',
       'Proyecto', 'Etapa', 'Productos', 'Presencial', 'Jefe Comercial',
       'Ejecutivo Comercial', 'Total Productos', 'Descuentos',
       'Valor Final Venta', 'RUT', 'Nombre', 'Apellido 1', 'Apellido 2',
       'Nombre Completo', 'Telefono', 'Celular', 'Direccion', 'Region',
       'Provincia', 'Comuna', 'Correo Electronico', 'Sexo', 'Estado Civil',
       'Rango Edad', 'Fecha Nacimiento', 'Profesión', 'Actividad', 'Negocio',
       'Clean Rut', 'Valid Rut', 'Format Rut'],
      dtype='object')

In [75]:
# merge
cot_mod_cp = cot_mod.copy()
cot_mod_cp['Format Rut'] = [x.replace('K','k') for x in cot_mod_cp['Format Rut'].values]
cot_merge = pd.merge(cot_mod_cp, ruts_scrapped,how='left', left_on='Format Rut', right_on='RUT')
# print(cot_merge.columns)
columns = ['Format Rut','Valid Rut','Negocio','Nombre','Apellido 1','Name','Gender','Sexo',
           'Adress','Direccion', 'Region','Provincia', 'Comuna_x', 'Comuna_y']
cot_merge[columns][cot_merge['Sexo'].isnull()].head(100)

,Format Rut,Valid Rut,Negocio,Nombre,Nombre,Apellido 1,Apellido 1,Name,Gender,Sexo,Adress,Direccion,Region,Provincia,Comuna_x,Comuna_y
45,19.088.621-2,True,False,Camila,Camila,Esparza,Esparza,ESPARZA TORRES CAMILA ALEXANDRA,MUJ,NaN,LAGUNA LO GALINDO 784 VILLA ENSENADA,NaN,VIII Región del Bío-Bío,Concepción,Concepción,MEDIO CAMINO
48,16.760.393-9,True,False,Carmen,Carmen,Hidalgo,Hidalgo,HIDALGO MOLINA CARMEN ALEJANDRA,MUJ,NaN,MEMBRILLAR 680,NaN,VIII Región del Bío-Bío,Concepción,Concepción,PENCO
49,16.760.393-9,True,False,Carmen,Carmen,Hidalgo,Hidalgo,HIDALGO MOLINA CARMEN ALEJANDRA,MUJ,NaN,MEMBRILLAR 680,NaN,VIII Región del Bío-Bío,Concepción,Concepción,PENCO
53,17.996.088-5,True,False,Francisco,Francisco,Gaete,Gaete,GAETE REY FRANCISCO ENRIQUE,VAR,NaN,LOS CANELOS 61,NaN,VIII Región del Bío-Bío,Concepción,San Pedro de la Paz,SAN PEDRO DE LA PAZ
72,16.405.169-2,True,False,Sindy,Sindy,Sagredo,Sagredo,SAGREDO ORTIZ SINDY MARISEL,MUJ,NaN,CALLE 4 2658 FLORESTA 3,NaN,VIII Región del Bío-Bío,Concepción,Concepción,HUALPEN
73,16.405.169-2,True,False,Sindy,Sindy,Sagredo,Sagredo,SAGREDO ORTIZ SINDY MARISEL,MUJ,NaN,CALLE 4 2658 FLORESTA 3,NaN,VIII Región del Bío-Bío,Concepción,Concepción,HUALPEN
74,16.405.169-2,True,False,Sindy,Sindy,Sagredo,Sagredo,SAGREDO ORTIZ SINDY MARISEL,MUJ,NaN,CALLE 4 2658 FLORESTA 3,NaN,VIII Región del Bío-Bío,Concepción,Concepción,HUALPEN
81,9.848.973-8,True,False,Patricio,Patricio,Lizama,Lizama,LIZAMA ARRIAGADA PATRICIO SEBASTIAN,VAR,NaN,PJ 15 970 PB J NOE,NaN,VIII Región del Bío-Bío,Concepción,Concepción,ARICA
83,14.073.436-5,True,False,Alejandra,Alejandra,Garrido,Garrido,GARRIDO HOGER CRISTINA ALEJANDRA,MUJ,NaN,VI LOS ALAMOS 177,NaN,VIII Región del Bío-Bío,Concepción,Concepción,CHILLAN
92,18.411.473-9,True,False,Joaquin,Joaquin,Inzunza,Inzunza,INZUNZA HOFFMANN JOAQUIN PABLO,VAR,NaN,18 SEP. 122 PEDRO VALDIVIA A,NaN,VIII Región del Bío-Bío,Concepción,Concepción,CONCEPCION CENTRO


In [56]:
# print(cot_mod_cp[['Nombre','Format Rut']].head())
# print(ruts_scrapped[['Name','RUT']].head())
# print(cot_mod_cp['Format Rut'].values)

ruts_scrapped['RUT'] = [x.replace(' ','') for x in ruts_scrapped['RUT'].values]

df = pd.merge(cot_mod_cp[['Nombre','Format Rut']],ruts_scrapped[['Name','RUT']],how='outer',left_on='Format Rut',right_on='RUT')
# print(ruts_scrapped['RUT'].values)

df

,Nombre,Nombre,Format Rut,Name,RUT
0,Nelda Del Carmen,Nelda Del Carmen,14.273.551-2,ROJAS ULLOA NELDA DEL CARMEN,14.273.551-2
1,Cecilia,Cecilia,13.309.954-9,SEPULVEDA TORRES CECILIA ANDREA,13.309.954-9
2,Yeni,Yeni,14.583.354-k,NaN,NaN
3,Maria Mercedes,Maria Mercedes,7.188.659-k,NaN,NaN
4,John Rodrigo,John Rodrigo,13.377.606-0,FUENTES GAJARDO JOHN RODRIGO,13.377.606-0
5,John Rodrigo,John Rodrigo,13.377.606-0,FUENTES GAJARDO JOHN RODRIGO,13.377.606-0
6,John Rodrigo,John Rodrigo,13.377.606-0,FUENTES GAJARDO JOHN RODRIGO,13.377.606-0
7,John Rodrigo,John Rodrigo,13.377.606-0,FUENTES GAJARDO JOHN RODRIGO,13.377.606-0
8,John Rodrigo,John Rodrigo,13.377.606-0,FUENTES GAJARDO JOHN RODRIGO,13.377.606-0
9,Zarah,Zarah,16.863.328-9,LOBOS LILLO ZARAH VICTORIA,16.863.328-9


# 3.Construct Data

## 3.1 Nuevos Atributos

### 3.1.1.Edad
Variable construida a partir de la fecha de nacimiento

In [ ]:
# Construir variable 'Edad' a partir de fecha de Nacimiento
import datetime
now = datetime.datetime.now().year

fechas_nacimiento = pd.to_datetime(cot_all['Fecha Nacimiento']).apply(lambda x: x.date()).tolist()

def anio_nac(x):
    try:
        year = int(x.year)
        dif = now - year
        if dif >= 18:
            return dif
        else:
            return 0
    except (ValueError,AttributeError):
        return 0
        
    return r

edad = [anio_nac(x) for x in fechas_nacimiento]
cot_mod['edad'] = edad



In [ ]:
cot_mod = cot_mod.drop(['Fecha Nacimiento'], axis=1)

### 3.1.2.Descuento
Variable que indica si el cliente recibio descuento

In [ ]:
# Construir 'descuento': binario : [True, False]
recibio_descuento = cot_all['Descuentos'].tolist()

def is_desc(x):
    try:
        desc = int(x)
        if desc > 0:
            return True
        else:
            return False
    except ValueError:
        return False

descuento = [is_desc(x) for x in recibio_descuento]
cot_mod['recibio_descuento'] = descuento
# descuento

### 3.1.3.Cantidad de Productos

In [ ]:
# Construir 'productos': categorico : ['depto','casa','estacionamiento','bodega','Estudio']
productos = cot_all['Productos'].tolist()
depto = []; estacionamiento=[]; bodega = []; estudio = [];nan = 0
def get_products(x,word,numeric=False):
    w=0
    try:
        p =str(x)
        arr_p = x.split(',')
        for e in arr_p:
            if word in e:
                w+=1
#                 print(word + " :",e)
                break
            elif numeric and ('B' not in e or 'Est' not in e):
                try:
                    int(e)
#                     print(word + " :",e)
                    w+=1
                except ValueError:
                    pass
        return w
    except (ValueError,AttributeError):
        try:
            int(x)
            if numeric:
                return 1
            else:
                return 0
        except ValueError:
            return 0

cot_mod['#vivienda'] = [get_products(x,'T',numeric=True) for x in productos]
cot_mod['#bodega'] = [get_products(x,'Bod') for x in productos]
cot_mod['#estacionamiento'] = [get_products(x,'Est') for x in productos]
cot_mod['#estudio'] = [get_products(x,'Estudio') for x in productos]
cot_mod['#lan'] = [get_products(x,'lan') for x in productos]

In [ ]:
print(cot_mod[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].head())
print(cot_mod[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].tail())

In [ ]:
cot_mod = cot_mod.drop(['Productos'], axis=1)

### 3.1.4.Latitud y Longitud

In [ ]:
from geopy.geocoders import Nominatim

def get_latlong(direccion):
    geolocator = Nominatim()
    # direccion = "TUCAPEL 937 , CONCEPCION CENTRO"
    location = geolocator.geocode(direccion)
    print(location.address)
    # Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
    print((location.latitude, location.longitude))
    # (40.7410861, -73.9896297241625)
    print(location.raw)

get_latlong('')
# Constuir 'latitud' a partir de 'Direccion'
# Constuir 'longitud' a partir de 'Direccion'

### 3.1.6.Region
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [ ]:
cot_mod['Region'].unique()

In [ ]:
cot_mod['region'] = cot_all['Region'].copy()
cot_mod['region'] = cot_mod['region'].replace(
    {'VIII Región del Bío-Bío':'Bio-Bio',
     'XIII Región Metropolitana de Santiago':'Metropolitana'}
)
cot_mod['region'] = cot_mod['region'].replace(
    ['II Región de Antofagasta',
       'IV Región de Coquimbo', 'VII Región del Maule',
       'X Región de Los Lagos',
       'V Región de Valparaíso',
       'VI Región del Libertador General Bernardo O Higgins',
       'XIV Región de Los Ríos',
       'XI Región de Aysen del General Carlos Ibáñez del Campo',
       'I Región de Tarapacá', 'IX Región de La Araucanía',
       'XII Región de Magallanes y de La Antártica Chilena',
       'XV Region de Arica y Parinacota', 'III Región de Atacama'],'Otro'
)

In [ ]:
cot_mod['region'] = cot_mod['region'].astype('category')
cot_mod['region'].head()

In [ ]:
cot_mod = cot_mod.drop(['Region'], axis=1)

### 3.1.7.Provincia
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [ ]:
cot_all['Provincia'].unique()

In [ ]:
cot_mod['provincia'] = cot_all['Provincia'].copy()
cot_mod['provincia'] = cot_mod['provincia'].replace(
    {'Concepción':'Concepcion',
     'Santiago':'Santiago'}
)
cot_mod['provincia'] = cot_mod['provincia'].replace(
    ['Antofagasta', 'Limarí', 'Talca', 'Biobío', 'Ñuble',
       'Llanquihue','Maipo', 'Valparaíso', 'Cachapoal',
       'Arauco', 'Valdivia', 'Coyhaique', 'Linares', 'Osorno',
       'San Felipe de Aconcagua', 'Iquique', 'Cautín', 'Capitán Prat',
       'Aysén', 'Chacabuco', 'Melipilla', 'Magallanes', 'Malleco',
       'Arica ', 'Elqui', 'El Loa', 'Cauquenes', 'Colchagua', 'Valdivia ',
       'Copiapó', 'Última Esperanza', 'Cordillera', 'Curicó', 'Chiloé',
       'Choapa', 'Huasco','Arica','Isla de Pascua'],'Otro'
)

In [ ]:
cot_mod['provincia'] = cot_mod['provincia'].astype('category')
cot_mod['provincia'].head()

In [ ]:
cot_mod = cot_mod.drop(['Provincia'], axis=1)

### 3.1.8.Comuna
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [ ]:
cot_all['Comuna'].unique()

In [ ]:
cot_mod['comuna'] = cot_all['Comuna'].copy()

cot_mod['comuna'] = cot_mod['comuna'].replace(
    ['Ovalle', 'Constitución','Tomé', 'Chillán','Yungay',
     'Puerto Montt', 'Santiago', 'El Bosque', 'La Florida','Las Condes',
     'Ñuñoa', 'Providencia', 'Buin','Florida','Viña del Mar',
     'Machalí', 'Vitacura', 'Arauco', 'Contulmo', 'Concón',
     'Santa Juana', 'Puerto Varas', 'Cañete', 'Huechuraba', 'Valdivia',
     'Coyhaique', 'Colbún', 'Osorno', 'San Felipe', 'Iquique',
     'Rancagua', 'Lota', 'Temuco', 'Cochrane', 'Chillán Viejo', 'Maipú',
     'Aysén', 'Villarrica', 'La Reina', 'Peñalolén', 'Estación Central',
     'Colina', 'Quilicura', 'Curanilahue', 'Algarrobo', 'Talca',
     'San Carlos', 'Punta Arenas', 'Mulchén', 'Coelemu', 'Angol',
     'Laja', 'Nacimiento', 'Arica', 'La Serena', 'Hualqui',
     'San Miguel', 'Calama', 'Pelluhue', 'Quillón', 'Valparaíso',
     'Llanquihue', 'Los Álamos', 'Renca', 'Santa Cruz', 'Panguipulli',
     'Cabrero', 'Cerrillos', 'Pudahuel', 'Copiapó', 'Natales', 'Tirúa',
     'Cauquenes', 'La Cisterna', 'Yumbel', 'Ránquil', 'Padre Las Casas',
     'Las Cabras', 'Coquimbo', 'Lebu', 'San José de Maipo', 'San Ramón',
     'Curicó', 'Independencia', 'Lampa ', 'Castro', 'Punitaqui',
     'Conchalí', 'San Rosendo', 'Alto Hospicio', 'Illapel', 'Huasco',
     'Collipulli', 'Tucapel', 'Alto Biobío','Pirque','Lampa','Isla de Pascua', 'Arica'],'Otro'
)

In [ ]:
cot_mod['comuna'] = cot_mod['comuna'].astype('category')
cot_mod['comuna'].head()

In [ ]:
cot_mod = cot_mod.drop(['Comuna'], axis=1)

### 3.1.9.Atributos Provistos
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [ ]:
cot_mod['is_nombre'] = ~cot_all['Nombre'].isnull()
cot_mod['is_apellido1'] = ~cot_all['Apellido 1'].isnull()
cot_mod['is_apellido2'] = ~cot_all['Apellido 2'].isnull()
cot_mod['is_nombre_completo'] = ~cot_all['Nombre Completo'].isnull()
cot_mod['is_correo'] = ~cot_all['Correo Electronico'].isnull()
cot_mod['is_telefono'] = ~cot_all['Telefono'].isnull()
cot_mod['is_actividad'] = ~cot_all['Actividad'].isnull()
cot_mod['is_estado_civil'] = ~cot_all['Estado Civil'].isnull()

cot_mod = cot_mod.drop(['Telefono','Correo Electronico','Telefono','Nombre','Apellido 1','Apellido 2','Nombre Completo'], axis=1)

### 3.1.11.Negocio
Indica si el cliente siguio el proceso de cotizacion y lo transformo en un proceso de negocio

In [ ]:
# ruts de los clientes en negocio
neg_ids = neg_all['ID Cotizacion'].unique().tolist()

# cot_neg = cot_all.copy()
cot_mod['Negocio'] = cot_all['ID'].isin(neg_ids)
cot_mod[cot_mod['Negocio'] == True].shape

In [ ]:
# negocio = []
# for rut in cot_all['RUT'].tolist():
#     if rut in neg_ruts:
# #         print(rut)
#         df = neg_all[neg_all['RUT']==rut]
#         negocio.append(df['Proyecto'])

# print(negocio)
# print(neg_all.shape[0])
# print("Nro Filas en cot que tienen ruts en neg :",cot_mod.loc[cot_neg['Negocio']==True].shape[0])

### 3.2. Extraccion de Datos
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [ ]:
# Construir '#cotizaciones' a partir del nro de filas donde participa el mismo rut
nro_cot_depto = []
nro_cot_esta = []
nro_cot_estu = []
nro_cot_bod = []

mean_cot_depto = []
mean_cot_esta = []
mean_cot_estu = []
mean_cot_bod = []

ruts = []
ruts_nro_cot = []
rut_reg = []
rut_prov = []
rut_com = []

nro_proyectos = []

negocio = []

cot_ruts = pd.DataFrame()

for group, frame in  cot_mod.groupby('RUT'):

    #Check , debe ir a data exploration
#     print(frame['Sexo'].unique())
    if len(frame['Proyecto'].unique()) > 1:
        print('nombre ',group, frame['Proyecto'].unique(), frame['Negocio'].unique(),frame['Jefe Comercial'].unique())
#     if len(frame['is_correo'].unique()) > 1:
#         print('correo ',group)
#         print(group)
#     if len(frame['Ejecutivo Comercial'].unique()) > 1:
#         print('JC ',group)
#         print(group)
    
    nro_proyectos.append(len(frame['Proyecto'].unique()))
    negocio.append(frame['Negocio'].tolist()[0])
    
    ruts.append(group)
    rut_reg.append(frame['region'].tolist()[0])
    rut_prov.append(frame['provincia'].tolist()[0])
    rut_com.append(frame['comuna'].tolist()[0])
    
    nro_cot_depto.append(frame['#vivienda'].sum())
    nro_cot_bod.append(frame['#bodega'].sum())
    nro_cot_esta.append(frame['#estacionamiento'].sum())
    nro_cot_estu.append(frame['#estudio'].sum())
    
    mean_cot_depto.append(frame['#vivienda'].mean())
    mean_cot_bod.append(frame['#bodega'].mean())
    mean_cot_esta.append(frame['#estacionamiento'].mean())
    mean_cot_estu.append(frame['#estudio'].mean())
    ruts_nro_cot.append(frame['ID'].count())

### 3.1.10.Tiempo entre Cotizaciones
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [ ]:
# Construir 'promedio_de_tiempo' entre cotizacones donde participa el mismo rut

# 4.Integrate Data

In [ ]:
list_cols = [ruts,rut_reg,rut_prov,rut_com,nro_proyectos,nro_cot_bod, nro_cot_depto, nro_cot_esta, nro_cot_estu, mean_cot_bod,
             mean_cot_depto,mean_cot_esta,mean_cot_estu, negocio]
list_labels = ['rut','loc_region','loc_provincia','loc_comuna','nro_proyectos','nro_cot_bod', 'nro_cot_depto', 'nro_cot_esta', 'nro_cot_estu', 'mean_cot_bod',
             'mean_cot_depto','mean_cot_esta','mean_cot_estu','negocio']

zipped = list(zip(list_labels,list_cols))
data = dict(zipped)
personas = pd.DataFrame(data)

personas

In [ ]:
personas.loc_provincia.unique()

In [ ]:
personas.info()

In [ ]:
# Transformar todo en un data set

# 5.Format Data

In [ ]:
personas['loc_comuna'] = personas['loc_comuna'].astype('category')
personas['loc_provincia'] = personas['loc_provincia'].astype('category')
personas['loc_region'] = personas['loc_region'].astype('category')

In [ ]:
personas.to_csv(data_directory+'personas.csv')